In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import Counter,defaultdict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [ ]:
train_set_ftb = json.load(open('./corpus/fr/fr.ftb.train.json', encoding = 'utf-8'))
test_set_ftb = json.load(open('./corpus/fr/fr.ftb.test.json', encoding = 'utf-8'))
train_set_gsd = json.load(open('./corpus/fr/fr.gsd.train.json', encoding = 'utf-8'))
test_set_gsd = json.load(open('./corpus/fr/fr.gsd.test.json', encoding = 'utf-8'))
train_set_partut = json.load(open('./corpus/fr/fr.partut.train.json', encoding = 'utf-8'))
test_set_partut = json.load(open('./corpus/fr/fr.partut.test.json', encoding = 'utf-8'))
train_set_pud = json.load(open('./corpus/fr/fr.pud.train.json', encoding = 'utf-8'))
test_set_pud = json.load(open('./corpus/fr/fr.pud.test.json', encoding = 'utf-8'))
train_set_sequoia = json.load(open('./corpus/fr/fr.sequoia.train.json', encoding = 'utf-8'))
test_set_sequoia = json.load(open('./corpus/fr/fr.sequoia.test.json', encoding = 'utf-8'))
train_set_spoken = json.load(open('./corpus/fr/fr.spoken.train.json', encoding = 'utf-8'))
test_set_spoken = json.load(open('./corpus/fr/fr.spoken.test.json', encoding = 'utf-8'))

test_set_foot = json.load(open('./corpus/fr/fr.foot.test.json', encoding = 'utf-8'))
test_set_natdis = json.load(open('./corpus/fr/fr.natdis.test.json', encoding = 'utf-8'))

In [3]:
def words_and_labels(data_set):
    
    words = []
    labels = []
    for sentence,label in data_set:
        for w,l in zip(sentence,label):
            words.append(w)
            labels.append(l)
    
    return words,labels

train_data,train_label = words_and_labels(train_set)

In [4]:
def ambiguous_words(train_data,train_label):
    
    words = defaultdict(lambda: set())
    ambiguous_words = []
    
    for word,label in zip(train_data,train_label):
        words[word].add(label)
    
    for word,label in zip(train_data,train_label):
        if(len(words[word]) > 1):
            ambiguous_words.append(word)
    
    return ambiguous_words

ambiguous_words = ambiguous_words(train_data,train_label)
print(len(ambiguous_words))

212787


In [5]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in train_data else 'nan' for w in sequence]

def simplify_decoding(X, model):
    
    """X should be a 1-D sequence of observations for the model to predict"""
    
    w_list = replace_unknown(X)
    
    id_oov = []
    id_amb = []
    
    _, state_path = model.viterbi(w_list)
    
    for i in range(len(w_list)):
        if w_list[i] == 'nan':
            id_oov.append(i)
            state_path_oov = state_path
        if w_list[i] in ambiguous_words:
            id_amb.append(i)
            state_path_amb = state_path
    
    state = []
    state_oov = []
    state_amb = []
    for i in range(len(state_path[1:-1])):
        state.append(state_path[i+1][1].name)
        if i in id_oov:
            state_oov.append(state_path[i+1][1].name)
        if i in id_amb:
            state_amb.append(state_path[i+1][1].name)
            
    return state,state_oov,id_oov,state_amb,id_amb

In [6]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    
    correct = total_predictions = 0
    correct_oov = total_predictions_oov = 0
    correct_amb = total_predictions_amb = 0
    for observations, actual_tags in zip(X, Y):
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags,most_likely_tags_oov,id_oov,most_likely_tags_amb,id_amb = simplify_decoding(observations, model)
            actual_tags_oov = list(actual_tags[i] for i in id_oov)
            actual_tags_amb = list(actual_tags[i] for i in id_amb)

            correct_oov += sum(p == t for p, t in zip(most_likely_tags_oov, actual_tags_oov))
            total_predictions_oov += len(most_likely_tags_oov)
            
            correct_amb += sum(p == t for p, t in zip(most_likely_tags_amb, actual_tags_amb))
            total_predictions_amb += len(most_likely_tags_amb)

            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
            total_predictions += len(most_likely_tags)
        except:
            pass
        
    return correct / total_predictions, correct_oov/total_predictions_oov, correct_amb/total_predictions_amb

In [7]:
def pair_counts(data_set):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    
    dict_label_word = defaultdict(list)
    dict_word = defaultdict(list)

    for sentence,labels in data_set:
        for word,label in zip(sentence,labels):
            dict_label_word[label].append(word)
    
    for k in dict_label_word.keys():
        dict_word[k] = Counter(dict_label_word[k])
        
    return dict_word

In [8]:
def starting_counts(data_set):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    
    dict_starting = defaultdict(list)
    
    lab = set()
    for _,labels in data_set:
        for label in labels:
            lab.add(label)
            
    for label in lab:
        dict_starting[label] = len([labels[0] for _,labels in data_set if labels[0]==label])
        
    return dict_starting

In [9]:
def ending_counts(data_set):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    
    dict_ending = defaultdict(list)
    
    lab = set()
    for _,labels in data_set:
        for label in labels:
            lab.add(label)
            
    for label in lab:
        dict_ending[label] = len([labels[-1] for _,labels in data_set if labels[-1]==label])
    return dict_ending

In [10]:
def unigram_counts(data_set):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    
    dict_tag = defaultdict(int)
    
    for _,labels in data_set:
        for label in labels:
            dict_tag[label] += 1           
    return dict_tag

In [11]:
def bigram_counts(data_set):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """

    dict_bigram = defaultdict(int)
    for _,labels in data_set:
        for i in range(1,len(labels)):
            dict_bigram[(labels[i-1],labels[i])] += 1
            
    return dict_bigram

In [74]:
train_set = train_set_ftb
test_set = test_set_foot

In [75]:
hmm = HiddenMarkovModel(name="base-hmm-tagger")

In [76]:
label_starts = starting_counts(train_set)
label_ends = ending_counts(train_set)
label_unigrams = unigram_counts(train_set)
label_bigrams = bigram_counts(train_set)

In [77]:
count_label_and_word = pair_counts(train_set)
states = {}
for label, word_dict in count_label_and_word.items(): #data.training_set.tagset
    p_words_given_label_state = defaultdict(float)
    # for each tag/word, calculate P(word|tag)
    for word in word_dict.keys(): # data.training_set.vocab
        p_words_given_label_state[word] = count_label_and_word[label][word] / label_unigrams[label] 
    # create a new state for each tag from the dict of words that represents P(word|tag)
    emission = DiscreteDistribution(dict(p_words_given_label_state))
    states[label] = State(emission, name=label)
    
hmm.add_states(list(states.values()))

In [78]:
label_list = set()
for _,labels in train_set:
    for label in labels:
        label_list.add(label)

In [79]:
# Adding start states
for label in label_list:        
    state = states[label]
    hmm.add_transition(hmm.start, state, label_starts[label]/len(label_list))

# Adding end states
for label in label_list: 
    state = states[label]
    hmm.add_transition(state, hmm.end, label_ends[label]/label_unigrams[label])

# Adding pairs
for label1 in label_list: 
    state1 = states[label1]
    for label2 in label_list: 
        state2 = states[label2]
        hmm.add_transition(state1, state2, label_bigrams[(label1, label2)]/label_unigrams[label1])

In [80]:
hmm.bake()

In [81]:
def sentences_and_labels(data_set):
    
    sentences = []
    labels = []
    for sentence,label in data_set:
        sentences.append(sentence)
        labels.append(label)
    
    return sentences,labels

test_data,test_label = sentences_and_labels(test_set)

In [82]:
hmm_training_acc = accuracy(test_data,test_label,hmm)

In [83]:
print(hmm_training_acc)

(0.6304284887087435, 0.14045669969840585, 0.8157717366883847)
